In [20]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")
X = {}
y = {}
X["train"] = np.load("X_train.npy")
y["train"] = np.load("y_train.npy")
X["dev"] = np.load("X_test.npy")
y["dev"] = np.load("y_test.npy")
X_text_train = np.load("X_train_text.npy")
X_text_dev = np.load("X_test_text.npy")
X["train"].shape, X["dev"].shape

((18000, 300), (1000, 300))

In [21]:
from sklearn.linear_model import LogisticRegression
l2 = 1
model = LogisticRegression(penalty='l2', C=1/l2)
model.fit(X["train"], y["train"])
model.score(X["dev"], y["dev"])

0.696

In [22]:
w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))], axis=1) # Concatenating one to calculate the gradient with respect to intercept
F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

gradient_train = F_train * error_train[:, None]
gradient_dev = F_dev * error_dev[:, None]
gradient_train.shape, gradient_dev.shape

((18000, 301), (1000, 301))

In [23]:
#from scipy import sparse
probs = model.predict_proba(X["train"])[:, 1]
hessian = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + l2 * np.eye(F_train.shape[1]) / X["train"].shape[0]
inverse_hessian = np.linalg.inv(hessian)

In [24]:
eps = 1 / X["train"].shape[0]
delta_k = -eps * inverse_hessian @ gradient_train.T
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
grad_f = F_dev * (pred * (1 - pred))
delta_pred = grad_f @ delta_k
delta_pred.shape

(1000, 18000)

In [25]:
np.save("old_predictions.npy", pred)

In [26]:
def Remove(k, scores, test_idx):
    #print("test_idx", test_idx)
    #print("old")
    #print(pred[test_idx])
    
    if pred[test_idx] > 0.5:
        top_k_index = scores[test_idx].argsort()[-k:]
    else:
        top_k_index = scores[test_idx].argsort()[:k]

    X_k = np.delete(X["train"], top_k_index, axis=0)
    y_k = np.delete(y["train"], top_k_index, axis=0)

        
    prediction = -np.sum(scores[test_idx][top_k_index])
    print("prediction", prediction)

    return X_k, y_k, prediction

In [27]:
ls

CSV_emo_LR_alg1.csv
CSV_speech_LR_alg1.csv
CSV_speech_LR_alg1.ipynb
Data_loader.ipynb
Sentiment-BNB.pickle
Smallest_k_valid.ipynb
Untitled.ipynb
Use_approK.ipynb
X_test.npy
X_test_text.npy
X_train.npy
X_train_text.npy
appro_ks.npy
essay_data/
new_pred.npy
npy2tsv.ipynb
old_predictions.npy
training.1600000.processed.noemoticon.csv
twitter-sentiment-analysis-for-beginners.ipynb
y_test.npy
y_train.npy
y_train_text.npy


In [28]:
app_k = np.array(np.load("appro_ks.npy", allow_pickle=True).squeeze())
app_k_nonon=[]
for i in range(X["dev"].shape[0]):
    print(app_k[i])
    if (not app_k[i] > 0):
        app_k_nonon.append(X["train"].shape[0])
    else:
        app_k_nonon.append(app_k[i])

145.0
84.0
301.0
35.0
27.0
221.0
8.0
7.0
1009.0
383.0
77.0
16.0
29.0
459.0
50.0
135.0
18.0
742.0
75.0
134.0
8.0
510.0
2177.0
4.0
502.0
12.0
148.0
78.0
128.0
53.0
1106.0
244.0
141.0
72.0
71.0
4.0
1114.0
531.0
9.0
112.0
356.0
23.0
440.0
320.0
66.0
2.0
30.0
95.0
56.0
8.0
275.0
74.0
132.0
48.0
276.0
2968.0
24.0
3.0
409.0
112.0
29.0
521.0
399.0
8.0
34.0
128.0
194.0
2738.0
63.0
2566.0
28.0
66.0
35.0
3.0
123.0
2.0
48.0
9.0
143.0
87.0
39.0
292.0
94.0
325.0
57.0
82.0
2074.0
42.0
84.0
42.0
33.0
30.0
671.0
107.0
84.0
16.0
1.0
91.0
1.0
434.0
6.0
1142.0
186.0
177.0
107.0
10.0
70.0
43.0
57.0
131.0
439.0
34.0
171.0
121.0
224.0
209.0
7.0
24.0
2592.0
275.0
191.0
39.0
1.0
18.0
7.0
45.0
217.0
47.0
100.0
1875.0
549.0
525.0
160.0
19.0
224.0
29.0
91.0
604.0
28.0
81.0
1117.0
385.0
226.0
115.0
402.0
120.0
135.0
20.0
5.0
227.0
69.0
497.0
90.0
71.0
18.0
51.0
2.0
259.0
766.0
18.0
369.0
2208.0
34.0
139.0
627.0
358.0
637.0
594.0
1738.0
6.0
11.0
19.0
15.0
1201.0
12.0
3070.0
106.0
64.0
331.0
22.0
108.0
203.0
nan
8.0

In [29]:
sort_dev_k = np.array(app_k_nonon).argsort()
sort_dev_k

array([224, 741, 863, 316, 398, 367, 764, 689, 671, 988, 122, 268, 520,
       993,  98, 378, 438, 874, 552,  96, 837,  45, 687, 692, 728, 614,
       605,  75, 835, 303, 234, 156, 313, 625, 434, 903, 894, 804,  73,
       526, 740, 370, 514, 965,  57, 364, 586, 710, 682, 600, 373,  35,
       594, 232, 645, 976, 358, 319, 798,  23, 850, 707, 584, 277, 849,
       908, 777, 604, 815, 759, 655, 524, 148, 215, 770, 169, 100, 760,
       437, 831, 889, 238, 577, 709, 124, 395, 116, 960, 924, 236, 657,
       357, 634, 440, 441, 733,   7, 904, 408, 755, 885,  20,  49, 765,
       391, 984,   6, 950, 183, 472,  63, 413, 264, 753, 836, 758,  38,
       467, 666,  77, 554, 197, 519, 425, 266, 289, 869, 931, 399, 105,
       360, 732, 292, 287, 286, 170, 330, 446, 573, 226, 662, 841, 445,
       174, 963, 537, 482, 827, 724,  25, 388, 726, 712, 902, 534, 458,
       893, 324, 172, 675, 504, 196, 632,  95,  11, 937, 567, 550, 939,
       597, 638, 649, 665, 668, 693, 734, 154,  16, 886, 539, 12

In [30]:
app_k_nonon[sort_dev_k[0]]

1.0

In [31]:
header = ["test_movie_id", "test_movie_review_text", "old_prediction", "right_prediction", "k", "successful_flip"]
for i in range(X["train"].shape[0]):
    header.append("train_example_" + str(i+1))

In [32]:
new_predictions = np.load("new_pred.npy", allow_pickle=True)

In [33]:
data = []
for test_movie_id in sort_dev_k:
    
    test_point = []
    # id
    test_point.append(test_movie_id)
    # test_movie_review_text
    test_point.append("label: " + str(y["dev"][test_movie_id]) + "  doc: " + str(X_text_dev[test_movie_id]))
    test_point.append(pred[test_movie_id].item())
    test_point.append(np.round(pred[test_movie_id].item()) == y["dev"][test_movie_id].item())
    # k
    k = app_k[test_movie_id]
    test_point.append(str(k))
    print("k", k)
    if not (k >= 0):
        test_point.append(None)
        data.append(test_point)
        continue
    k = int(k)
    test_point.append(np.round(new_predictions[test_movie_id].item()) != np.round(pred[test_movie_id].item()))
        
    
    if pred[test_movie_id] > 0.5:
        top_k_index = delta_pred[test_movie_id].argsort()[-k:]
    else:
        top_k_index = delta_pred[test_movie_id].argsort()[:k]
    
    # training examples
    for idx in top_k_index:
        test_point.append("id: " + str(idx) + "  label: " + str(y["train"][idx]) + "  doc: " + str(X_text_train[idx]))
    data.append(test_point)

k 1.0
k 1.0
k 1.0
k 1.0
k 1.0
k 1.0
k 1.0
k 1.0
k 1.0
k 1.0
k 1.0
k 1.0
k 1.0
k 1.0
k 1.0
k 1.0
k 1.0
k 1.0
k 1.0
k 1.0
k 2.0
k 2.0
k 2.0
k 2.0
k 2.0
k 2.0
k 2.0
k 2.0
k 2.0
k 2.0
k 2.0
k 2.0
k 2.0
k 2.0
k 3.0
k 3.0
k 3.0
k 3.0
k 3.0
k 3.0
k 3.0
k 3.0
k 3.0
k 3.0
k 3.0
k 3.0
k 3.0
k 4.0
k 4.0
k 4.0
k 4.0
k 4.0
k 4.0
k 4.0
k 4.0
k 4.0
k 4.0
k 4.0
k 4.0
k 4.0
k 5.0
k 5.0
k 5.0
k 5.0
k 5.0
k 5.0
k 5.0
k 5.0
k 5.0
k 5.0
k 5.0
k 5.0
k 5.0
k 5.0
k 6.0
k 6.0
k 6.0
k 6.0
k 6.0
k 6.0
k 6.0
k 6.0
k 6.0
k 6.0
k 7.0
k 7.0
k 7.0
k 7.0
k 7.0
k 7.0
k 7.0
k 7.0
k 7.0
k 7.0
k 7.0
k 7.0
k 7.0
k 7.0
k 7.0
k 8.0
k 8.0
k 8.0
k 8.0
k 8.0
k 8.0
k 8.0
k 8.0
k 8.0
k 8.0
k 8.0
k 8.0
k 8.0
k 9.0
k 9.0
k 9.0
k 9.0
k 9.0
k 9.0
k 9.0
k 9.0
k 10.0
k 10.0
k 10.0
k 10.0
k 10.0
k 10.0
k 10.0
k 10.0
k 10.0
k 10.0
k 10.0
k 11.0
k 11.0
k 11.0
k 11.0
k 11.0
k 11.0
k 11.0
k 11.0
k 11.0
k 11.0
k 11.0
k 12.0
k 12.0
k 12.0
k 12.0
k 12.0
k 12.0
k 12.0
k 12.0
k 12.0
k 13.0
k 13.0
k 14.0
k 14.0
k 14.0
k 14.0
k 15.0
k 15.0
k 15.0


In [19]:
    test_movie_id = 4
    test_point = []
    # id
    test_point.append(test_movie_id)
    # test_movie_review_text
    test_point.append("label: " + str(y["dev"][test_movie_id]) + "  doc: " + str(X_text_dev[test_movie_id]))
    test_point.append(pred[test_movie_id].item())
    test_point.append(np.round(pred[test_movie_id].item()) == y["dev"][test_movie_id].item())
    # k
    k = app_k[test_movie_id]
    test_point.append(str(k))
    print("k", k)
    if not (k >= 0):
        test_point.append(None)
        data.append(test_point)
    k = int(k)
    test_point.append(np.round(new_predictions[test_movie_id].item()) != np.round(pred[test_movie_id].item()))
        
    
    if pred[test_movie_id] > 0.5:
        top_k_index = delta_pred[test_movie_id].argsort()[-k:]
    else:
        top_k_index = delta_pred[test_movie_id].argsort()[:k]
    
    # training examples
    for idx in top_k_index:
        test_point.append("id: " + str(idx) + "  label: " + str(y["train"][idx]) + "  doc: " + str(X_text_train[idx]))
    data.append(test_point)
    print("top_k_index", top_k_index)

k 27.0
top_k_index [13085 16925  5393 15937 17031  9848  2036 17755  6309  8156   594  1745
  7810 10500  1215  8762 11015  3735 11352  2020  2498 14779 14526  7381
 13514  9250 11096]


In [34]:
import csv

with open('CSV_speech_LR_alg1.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)


In [35]:
ls

CSV_emo_LR_alg1.csv
CSV_speech_LR_alg1.csv
CSV_speech_LR_alg1.ipynb
Data_loader.ipynb
Sentiment-BNB.pickle
Smallest_k_valid.ipynb
Untitled.ipynb
Use_approK.ipynb
X_test.npy
X_test_text.npy
X_train.npy
X_train_text.npy
appro_ks.npy
essay_data/
new_pred.npy
npy2tsv.ipynb
old_predictions.npy
training.1600000.processed.noemoticon.csv
twitter-sentiment-analysis-for-beginners.ipynb
y_test.npy
y_train.npy
y_train_text.npy


In [44]:
pwd

'/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/LR/IF/Logistic_SST/IF_lr_fix'